In [15]:
import pandas as pd
import datetime
import numpy as np

# Linkedin Data from Kaggle

In [19]:
data = pd.read_csv("job postings 2023 24/postings.csv")

In [68]:
data.columns

Index(['job_id', 'company_name', 'title', 'description', 'max_salary',
       'pay_period', 'location', 'company_id', 'views', 'med_salary',
       'min_salary', 'formatted_work_type', 'applies', 'original_listed_time',
       'remote_allowed', 'job_posting_url', 'application_url',
       'application_type', 'expiry', 'closed_time',
       'formatted_experience_level', 'skills_desc', 'listed_time',
       'posting_domain', 'sponsored', 'work_type', 'currency',
       'compensation_type', 'normalized_salary', 'zip_code', 'fips'],
      dtype='object')

In [91]:
def convert_unix_to_ddmmyyyy(unix_time):
    # Convert from milliseconds to seconds
    unix_time_seconds = unix_time / 1000.0
    
    # Convert to datetime and handle errors
    formatted_dates = pd.to_datetime(unix_time_seconds, unit='s', errors='coerce')
    
    # Format as 'dd-mm-yyyy'
    return formatted_dates.dt.strftime('%d-%m-%Y')

In [92]:
raw_data = data.copy()

In [93]:
raw_data['original_listed_time_mod'] = convert_unix_to_ddmmyyyy(raw_data['original_listed_time'])

In [94]:
raw_data['original_listed_time_mod'] =pd.to_datetime(raw_data['original_listed_time_mod'], format='%d-%m-%Y', errors='coerce')

In [97]:
raw_data['original_listed_time_mod'].min()

Timestamp('2023-12-05 00:00:00')

In [113]:
raw_data['original_listed_time_mod'].max()

Timestamp('2024-04-20 00:00:00')

# Data Scraped from ScrapingDog API

In [112]:
import requests

def fetch_all_linkedin_jobs(api_key, fields, geoids, pages):
    url = "https://api.scrapingdog.com/linkedinjobs/"
    all_data = []  # Store data from all requests

    # Iterate over each combination of field, geoid, and page
    for field in fields:
        for geoid in geoids:
            for page in pages:
                params = {
                    "api_key": api_key,
                    "field": field,
                    "geoid": geoid,
                    "page": page
                }
                
                response = requests.get(url, params=params)
                
                if response.status_code == 200:
                    # Append data from each request to the list
                    all_data.extend(response.json())
                else:
                    print(f"Request failed for field: {field}, geoid: {geoid}, page: {page} with status code: {response.status_code}")

    return all_data

# Usage example
api_key = "671da7467fddaf7ca053001a"
fields = ["java", "data science", "machine learning"]
geoids = ["100293800","103736294","102277331"]  # NV, Colorado, California
pages = ["1", "2", "3"]

data = fetch_all_linkedin_jobs(api_key, fields, geoids, pages)



In [111]:
if data:
    new_Data = pd.DataFrame(data)
    new_Data.to_csv("261024_Job_Data.csv")